In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, shape, Point
import pyproj
from functools import partial
from shapely.ops import transform
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model
from sklearn import svm


%matplotlib inline

ModuleNotFoundError: No module named 'descartes'

In [ ]:
import pyproj
from functools import partial
from shapely.ops import transform


In [ ]:
cost_paths_cars = pd.read_csv('cost_paths_cars.csv')
cost_paths_cars = cost_paths_cars.drop(columns = ['Unnamed: 0'])
cost_paths_cars.head()

In [ ]:
equips_map = gpd.read_file('../pacote_USP/equips.shp')
equips_map.head()

In [ ]:
equips_map.crs

In [ ]:
fig,ax = plt.subplots(figsize = (15,15))
equips_map.plot(ax=ax);

In [ ]:
def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo

In [ ]:
sp_zones = gpd.read_file('../Shape/Zonas_2017_region.shp')
sp_zones = sp_zones[sp_zones.NomeMunici == 'São Paulo']
sp_zones.head()

In [ ]:
sp_zones.crs = {'init': 'epsg:22523'}
sp_zones = sp_zones.to_crs({'init': 'epsg:4326'})

In [ ]:
fig,ax = plt.subplots(figsize = (15,15))
sp_zones.plot(ax=ax);


In [ ]:
equips_within_zones = gpd.sjoin(equips_map, sp_zones, how="inner", op='intersects')
equips_within_zones = equips_within_zones[['id','geometry','NumeroZona']]
equips_within_zones.head()

In [ ]:
equips_merged = pd.merge(cost_paths_cars,equips_within_zones, how = 'inner', left_on = 'last', \
                         right_on = 'id', right_index = False, left_index = False
                        )
equips_merged = equips_merged.rename(columns = { 'first':'fid', \
                                                'last':'lid','cat1':'vol', \
                                                'NumeroZona':'lzone'})[['geometry','fid','lid','vol','lzone','cost_f']]
equips_merged.head()

In [ ]:
fig,ax = plt.subplots(figsize = (15,15))
sp_zones.plot(ax=ax, color = 'white', alpha = 0.25, linewidth=0.95, edgecolor='0.05',);
equips_within_zones.plot(ax=ax, markersize=5, color= 'green', marker ='o', label = 'Equipamento');
plt.axis('off');
plt.legend(prop = {'size' : 15});

In [ ]:
od_survey = pd.read_csv('../pesquisa_OD.csv', sep =';').dropna(how = 'all').replace('-',0)

od_survey

In [ ]:
equips_zones_data = pd.merge(equips_merged,od_survey, how = 'inner', left_on = 'lzone', \
                         right_on = 'zona', right_index = False, left_index = False
                        )

equips_zones_data

In [ ]:
n= 10083
equips_by_first = equips_zones_data[equips_zones_data.fid == n]

equips_by_first = equips_by_first.drop(columns = ['geometry','fid','lid','lzone','zona','nome','municipio'])

equips_by_first = equips_by_first.apply(pd.to_numeric, errors='ignore')

equips_by_first = equips_by_first[equips_by_first.vol > 0]


equips_by_first

corr = equips_by_first.corr()
corr_cols = list(corr['vol'].apply(lambda x : x if x > 0 else -x).sort_values(ascending=False)[1:11].index)
corr_cols

In [ ]:
X = equips_by_first[corr_cols]
y = equips_by_first[['vol']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)




In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression(normalize = True)]

trainingData    = X_train
trainingScores  = y_train
predictionData  = X_test

for item in classifiers:
    print(item)
    clf = item
    clf.fit(trainingData, trainingScores.values.ravel())
    predictionScore = clf.predict(predictionData)
    r2 = r2_score(y_test,predictionScore)
    print(r2)

In [ ]:

trainingData    = X_train
trainingScores  = y_train
predictionData  = X_test


clf = linear_model.TheilSenRegressor()
clf.fit(trainingData, trainingScores.values.ravel())
predictionScore = clf.predict(predictionData)
r2 = r2_score(y_test,predictionScore)


In [ ]:
m = pd.DataFrame(y_test)
m['vol_calc'] = predictionScore
m.head()

In [ ]:
#TODO Aplicar modelo
lm_model.coef_
